# Examen ETL: SPARK 02/02

* Autor: Beltrán Aller López
* Fecha: 02/12/2019
* Asignatura: ETL

Se podrá utilizar toda la información que se encuentra en el campus. 

El fichero de datos sobre el que se trabajará es el de partidosLigaNBA.txt.

A cada una de las preguntas hay que responder explicando brevemente que se pretende hacer antes de lanzar el código.

Al documento lo llamareís con vuestro nombre y apellido. Debeís enviarlo a mi correo de CUNEF antes del final del examen.

El lenguaje para trabajar con Spark podrá ser python o R indistintamente.

## Primera pregunta: Describe brevemente que diferencia el persists, cache y collect en spark. Explica brevemente casos en los que es interesante su aplicación

Cache es una transformación. Se utiliza para guardar transformaciones intermedias y así poder reutilizarlas posteriormente.

Persists es una tranformación también, al igual que cache. Se utiliza para lo mismo que cache, pero en este caso te permite pasarle como parámetro el nivel de almacenamiento.

Collect es una acción. Devuelve todos los elementos de un RDD en una lista de Python.

## Segunda pregunta: Explica brevemente los pasos a seguir para realizar la carga de un conjunto de datos (pasos que se siguieron en la práctica con datos de logs)

En primer lugar se ha de inicializar el contexto spark.

In [1]:
from pyspark import SparkContext
sc = SparkContext("local", "First App")

Ahora cargamos el archivo propiamente dicho

In [2]:
data_file = "./partidosLigaNBA.csv"
liga_data = sc.textFile(data_file)

Realizamos un conteo  a ver si están todos los datos

In [3]:
liga_data.count()

12908

Por último echamos un vistazo a ver la forma que tiene

In [4]:
liga_data.take(3)

['Date:Start..ET.:Visitor.Neutral:PTS:Home.Neutral:PTS.1',
 'Tue, Oct 30, 2007:"7:30 pm":Utah Jazz:117:Golden State Warriors:96',
 'Tue, Oct 30, 2007:"7:30 pm":Houston Rockets:95:Los Angeles Lakers:93']

## Tercera Pregunta: Índica un tipo de problema que puede empeorar los datos. (pe. Que no exista un representante del CDO en todas las áreas de negocio), pon algún ejemplo específico (pe. Datos duplicados) y cómo lo tratarías con técnicas de data cleaning.

Que estos no estén separados por los mismos caractéres siempre, por ejemplo, que a veces estén separados por comas y otras por puntos y comas. También que la identación varíe. Habría que establecer tantas expresiones regulares como casos y pasea el fichero con esas expresiones.

## Cuarta tarea: Inicializar spark context y cargar los datos desde el fichero.

In [7]:
from pyspark import SparkContext
sc = SparkContext("local", "First App")

In [9]:
data_file = "./partidosLigaNBA.csv"
liga_data = sc.textFile(data_file)

In [10]:
liga_data.take(3)

['Date:Start..ET.:Visitor.Neutral:PTS:Home.Neutral:PTS.1',
 'Tue, Oct 30, 2007:"7:30 pm":Utah Jazz:117:Golden State Warriors:96',
 'Tue, Oct 30, 2007:"7:30 pm":Houston Rockets:95:Los Angeles Lakers:93']

In [11]:
liga_data.count()

12908

Quito el header y los playoffs

In [12]:
liga_data_split = liga_data.filter(lambda x: "PTS" not in x).filter(lambda x: "Playoffs" not in x)

In [13]:
import re

Defino las expresiones regulares, hay 3 casos porque hay un equipo que tiene números en el nombre, por ello hay que considerar que ninguno los tenga, que los tenga el visitante o que los tenga el local.

In [14]:
expresion1 = '^(\S+)\,\s(\S+)\s(\d+)\,\s(\d{4})\:"(\d+[:]\d{2}\s\S+)"\:(\D+)\:(\d+)\:(\D+)\:(\d+)'

In [15]:
expresion2 = '^(\S+)\,\s(\S+)\s(\d+)\,\s(\d{4})\:"(\d+[:]\d{2}\s\S+)"\:(\D+\s\S+)\:(\d+)\:(\D+)\:(\d+)'

In [16]:
expresion3 = '^(\S+)\,\s(\S+)\s(\d+)\,\s(\d{4})\:"(\d+[:]\d{2}\s\S+)"\:(\D+)\:(\d+)\:(\D+\s\S+)\:(\d+)'

In [17]:
def parse_log(line):
    match = re.search(expresion1, line)
    if match is None:
        match = re.search(expresion2, line)
    if match is None:
        match = re.search(expresion3, line)
    return match.groups()

In [18]:
ejemplo = liga_data_split.take(2)

In [19]:
ejemplo[1]

'Tue, Oct 30, 2007:"7:30 pm":Houston Rockets:95:Los Angeles Lakers:93'

In [20]:
prueba = re.search(expresion1, ejemplo[1])

In [21]:
prueba.groups()

('Tue',
 'Oct',
 '30',
 '2007',
 '7:30 pm',
 'Houston Rockets',
 '95',
 'Los Angeles Lakers',
 '93')

## Quinta tarea: Media de la diferencia de puntos por año

Lo primero, aplicar el parseo.

In [22]:
liga_parsed = liga_data_split.map(lambda line:parse_log(line))

In [23]:
liga_parsed.take(1)

[('Tue',
  'Oct',
  '30',
  '2007',
  '7:30 pm',
  'Utah Jazz',
  '117',
  'Golden State Warriors',
  '96')]

Calculo la diferencia de puntos por partido, me quedaré con el año en que se jugó el partido y esa diferencia.

In [24]:
diferencia_puntos = liga_parsed.map(lambda x: (x[3], abs((int(x[6])-int(x[8])))))
# diferencia de puntos entre visitantes y locales

In [25]:
diferencia_puntos.take(10)

[('2007', 21),
 ('2007', 2),
 ('2007', 9),
 ('2007', 18),
 ('2007', 17),
 ('2007', 9),
 ('2007', 3),
 ('2007', 9),
 ('2007', 14),
 ('2007', 19)]

Ahora agrupo en función del año.

In [26]:
# ahora sumo todas las diferencias, de forma que tengo el año y la diferencia en valor absoluto
diferencia_agrupada = diferencia_puntos.reduceByKey(lambda x,y: x + y)
diferencia_agrupada.collect()

[('2017', 9172),
 ('2013', 14659),
 ('2016', 15397),
 ('2011', 9435),
 ('2014', 14547),
 ('2008', 15376),
 ('2007', 5060),
 ('2015', 14720),
 ('2009', 14595),
 ('2010', 14358),
 ('2012', 15986)]

Calculo el número de partidos que se han jugado por año. El método es parecido al de antes, pero hago un conteo.

In [27]:
# numero de partidos
num_partidos = liga_parsed.map(lambda x: (x[3],1)).reduceByKey(lambda x,y: x + y)
num_partidos.collect()

[('2017', 803),
 ('2013', 1324),
 ('2016', 1333),
 ('2011', 885),
 ('2014', 1334),
 ('2008', 1332),
 ('2007', 456),
 ('2015', 1319),
 ('2009', 1316),
 ('2010', 1321),
 ('2012', 1474)]

Ahora realizo un join de los dos rdd, diferencia de puntos y número de partidos, divido la diferencia entre el número de partidos.

In [28]:
# uno los rdd
diferencia_agrupada.join(num_partidos).map(lambda x:(x[0],(x[1][0]/x[1][1]))).sortByKey().collect()

[('2007', 11.096491228070175),
 ('2008', 11.543543543543544),
 ('2009', 11.090425531914894),
 ('2010', 10.86903860711582),
 ('2011', 10.661016949152541),
 ('2012', 10.845318860244234),
 ('2013', 11.071752265861027),
 ('2014', 10.9047976011994),
 ('2015', 11.159969673995452),
 ('2016', 11.550637659414853),
 ('2017', 11.422166874221668)]

## Sexta tarea: ¿Han jugado todos los equipos el mismo número de partidos? ¿ Si es qué no a que puede deberse?

Hago un conteo de los partidos como local de cada equipo.

In [29]:
num_partidos_equipo_local = liga_parsed.map(lambda x: (x[5],1)).reduceByKey(lambda y,z: y + z)

In [30]:
num_partidos_equipo_local.collect()

[('New Orleans Pelicans', 166),
 ('Sacramento Kings', 402),
 ('Charlotte Hornets', 127),
 ('Phoenix Suns', 413),
 ('San Antonio Spurs', 466),
 ('Miami Heat', 456),
 ('Golden State Warriors', 440),
 ('Brooklyn Nets', 218),
 ('Milwaukee Bucks', 414),
 ('New Jersey Nets', 197),
 ('Seattle SuperSonics', 41),
 ('Utah Jazz', 424),
 ('Los Angeles Lakers', 447),
 ('Indiana Pacers', 434),
 ('Houston Rockets', 434),
 ('Dallas Mavericks', 436),
 ('Oklahoma City Thunder', 408),
 ('Washington Wizards', 423),
 ('Cleveland Cavaliers', 452),
 ('Chicago Bulls', 437),
 ('Toronto Raptors', 424),
 ('Portland Trail Blazers', 428),
 ('New Orleans Hornets', 249),
 ('Memphis Grizzlies', 434),
 ('Denver Nuggets', 424),
 ('Philadelphia 76ers', 418),
 ('New York Knicks', 413),
 ('Orlando Magic', 432),
 ('Boston Celtics', 463),
 ('Atlanta Hawks', 448),
 ('Detroit Pistons', 414),
 ('Minnesota Timberwolves', 402),
 ('Charlotte Bobcats', 283),
 ('Los Angeles Clippers', 430)]

Hago lo mismo que antes pero para visitantes.

In [31]:
num_partidos_equipo_visitante = liga_parsed.map(lambda x: (x[7],1)).reduceByKey(lambda y,z: y + z)

In [32]:
num_partidos_equipo_visitante.collect()

[('New Orleans Pelicans', 166),
 ('Sacramento Kings', 402),
 ('Atlanta Hawks', 448),
 ('Charlotte Hornets', 126),
 ('Phoenix Suns', 412),
 ('San Antonio Spurs', 467),
 ('Golden State Warriors', 445),
 ('Brooklyn Nets', 217),
 ('Toronto Raptors', 426),
 ('New Jersey Nets', 197),
 ('Boston Celtics', 467),
 ('Utah Jazz', 422),
 ('Los Angeles Lakers', 450),
 ('Miami Heat', 461),
 ('Houston Rockets', 434),
 ('Dallas Mavericks', 431),
 ('Oklahoma City Thunder', 410),
 ('Washington Wizards', 421),
 ('Cleveland Cavaliers', 449),
 ('Chicago Bulls', 436),
 ('Milwaukee Bucks', 413),
 ('New Orleans Hornets', 250),
 ('Seattle SuperSonics', 41),
 ('Denver Nuggets', 424),
 ('Detroit Pistons', 415),
 ('Philadelphia 76ers', 416),
 ('New York Knicks', 412),
 ('Orlando Magic', 431),
 ('Indiana Pacers', 434),
 ('Charlotte Bobcats', 283),
 ('Portland Trail Blazers', 425),
 ('Minnesota Timberwolves', 402),
 ('Memphis Grizzlies', 433),
 ('Los Angeles Clippers', 431)]

Se observan diferencias entre algunos equipos, esto puede deberse a que hayan jugado playoff o no, además algunos equipos cambiaron de nombre y por tanto aparecen duplicados, con el nombre antiguo y el nuevo, como si fuesen distintos.

Una de las razones a las que se puede deber es si el equipo se clasificó o no para playoffs, aquellos que se clasifiquen y jueguen los playoffs habrán disputado más partidos

## Séptima pregunta: ¿Cuantos partidos ha ganado en Enero Cleveland?

Me quedo sólo con los partidos de enero.

In [33]:
partidos_enero = liga_parsed.filter(lambda x: x[1] == 'Jan')
partidos_enero.take(15)

[('Wed',
  'Jan',
  '2',
  '2008',
  '7:30 pm',
  'Houston Rockets',
  '93',
  'Boston Celtics',
  '97'),
 ('Wed',
  'Jan',
  '2',
  '2008',
  '7:00 pm',
  'Chicago Bulls',
  '109',
  'Charlotte Bobcats',
  '97'),
 ('Wed',
  'Jan',
  '2',
  '2008',
  '7:00 pm',
  'Atlanta Hawks',
  '94',
  'Cleveland Cavaliers',
  '98'),
 ('Wed',
  'Jan',
  '2',
  '2008',
  '7:30 pm',
  'Golden State Warriors',
  '99',
  'Dallas Mavericks',
  '121'),
 ('Wed',
  'Jan',
  '2',
  '2008',
  '7:00 pm',
  'Memphis Grizzlies',
  '90',
  'Indiana Pacers',
  '72'),
 ('Wed',
  'Jan',
  '2',
  '2008',
  '7:30 pm',
  'New Orleans Hornets',
  '95',
  'Los Angeles Clippers',
  '81'),
 ('Wed',
  'Jan',
  '2',
  '2008',
  '7:30 pm',
  'Milwaukee Bucks',
  '103',
  'Miami Heat',
  '98'),
 ('Wed',
  'Jan',
  '2',
  '2008',
  '7:00 pm',
  'Portland Trail Blazers',
  '90',
  'Minnesota Timberwolves',
  '79'),
 ('Wed',
  'Jan',
  '2',
  '2008',
  '7:30 pm',
  'Sacramento Kings',
  '107',
  'New York Knicks',
  '97'),
 ('We

De los de enero, me quedo con los que Cleveland es local y aparte con los que ganó.

In [34]:
partidos_enero_local_cleveland = partidos_enero.filter(lambda x: x[7] == 'Cleveland Cavaliers').filter(lambda x: int(x[8]) > int(x[6]))
partidos_enero_local_cleveland.count()

42

Ahora hago lo mismo que antes pero me quedo con sus victorias de visitante.

In [35]:
partidos_enero_visitante_cleveland = partidos_enero.filter(lambda x: x[5] == 'Cleveland Cavaliers').filter(lambda x: int(x[6]) > int(x[8]))
partidos_enero_visitante_cleveland.count()

41

Como local en enero ha ganado un total de 37 partidos entre 2007 y 2017 y, como visitante 56. Es decir, en estos 11 años ha ganado 93 partidos en enero.

## Octava pregunta: ¿Los Warrios son mejores fuera de casa o en casa?

Creo un filtro para qudarme con los Warriors como locales y sólo aquellos partidos ganados.

In [36]:
partidos_warriors_casa_victorias = liga_parsed.filter(lambda x: x[7] == 'Golden State Warriors').filter(lambda x: int(x[8]) > int(x[6]))
partidos_warriors_casa_victorias.count()

308

Hago lo mismo pero para visitante.

In [37]:
partidos_warriors_fuera_victorias = liga_parsed.filter(lambda x: x[5] == 'Golden State Warriors').filter(lambda x: int(x[6]) > int(x[8]))
partidos_warriors_fuera_victorias.count()

215

Fueron mejores fuera de casa

## Novena pregunta: Equipo que ha quedado primerio en victorias más temporadas. (si es que hay alguno que más)

Filtro para quedarme solo con las victorias de los locales y hago la suma de victorias por equipo.

In [38]:
mejor_local = liga_parsed.filter(lambda x: int(x[8]) > int(x[6])).map(lambda x: ((x[3],x[7]),1)).reduceByKey(lambda y,z: y + z)
mejor_local.collect()

[(('2013', 'Los Angeles Clippers'), 32),
 (('2013', 'Indiana Pacers'), 43),
 (('2012', 'Minnesota Timberwolves'), 22),
 (('2007', 'Atlanta Hawks'), 10),
 (('2007', 'Sacramento Kings'), 8),
 (('2017', 'Dallas Mavericks'), 15),
 (('2016', 'Brooklyn Nets'), 15),
 (('2007', 'Dallas Mavericks'), 14),
 (('2010', 'San Antonio Spurs'), 36),
 (('2011', 'New Orleans Hornets'), 17),
 (('2013', 'Portland Trail Blazers'), 23),
 (('2012', 'Cleveland Cavaliers'), 14),
 (('2014', 'Indiana Pacers'), 32),
 (('2014', 'Milwaukee Bucks'), 14),
 (('2010', 'Portland Trail Blazers'), 26),
 (('2010', 'New Orleans Hornets'), 25),
 (('2011', 'Boston Celtics'), 24),
 (('2007', 'San Antonio Spurs'), 16),
 (('2008', 'Memphis Grizzlies'), 17),
 (('2010', 'Indiana Pacers'), 26),
 (('2009', 'Chicago Bulls'), 30),
 (('2010', 'Cleveland Cavaliers'), 31),
 (('2010', 'Boston Celtics'), 36),
 (('2007', 'Orlando Magic'), 7),
 (('2016', 'Dallas Mavericks'), 19),
 (('2017', 'Phoenix Suns'), 9),
 (('2014', 'Oklahoma City Thund

Ordeno de mayor número de victorias a menor y obtengo los resultados.

In [39]:
mejor_local.sortBy(lambda x: -x[1]).collect()

[(('2012', 'Miami Heat'), 52),
 (('2008', 'Boston Celtics'), 51),
 (('2015', 'Golden State Warriors'), 51),
 (('2016', 'Golden State Warriors'), 49),
 (('2012', 'Oklahoma City Thunder'), 48),
 (('2009', 'Los Angeles Lakers'), 47),
 (('2013', 'Miami Heat'), 47),
 (('2016', 'Cleveland Cavaliers'), 46),
 (('2008', 'Los Angeles Lakers'), 45),
 (('2012', 'San Antonio Spurs'), 44),
 (('2013', 'Indiana Pacers'), 43),
 (('2013', 'San Antonio Spurs'), 42),
 (('2009', 'Cleveland Cavaliers'), 42),
 (('2008', 'Utah Jazz'), 42),
 (('2015', 'San Antonio Spurs'), 42),
 (('2009', 'Denver Nuggets'), 42),
 (('2014', 'San Antonio Spurs'), 42),
 (('2016', 'Toronto Raptors'), 41),
 (('2012', 'Los Angeles Clippers'), 41),
 (('2012', 'Memphis Grizzlies'), 40),
 (('2012', 'Boston Celtics'), 40),
 (('2008', 'Cleveland Cavaliers'), 40),
 (('2009', 'Orlando Magic'), 40),
 (('2015', 'Cleveland Cavaliers'), 40),
 (('2010', 'Los Angeles Lakers'), 39),
 (('2014', 'Los Angeles Clippers'), 39),
 (('2013', 'Denver Nugg

Hago lo mismo que hice para local para visitante.

In [40]:
mejor_visitante = liga_parsed.filter(lambda x: int(x[6]) > int(x[8])).map(lambda x: ((x[3],x[5]),1)).reduceByKey(lambda y,z: y + z)
mejor_visitante.sortBy(lambda x: -x[1]).collect()

[(('2012', 'San Antonio Spurs'), 38),
 (('2016', 'Golden State Warriors'), 38),
 (('2013', 'Miami Heat'), 38),
 (('2015', 'Golden State Warriors'), 37),
 (('2016', 'San Antonio Spurs'), 37),
 (('2009', 'Los Angeles Lakers'), 34),
 (('2009', 'Cleveland Cavaliers'), 34),
 (('2008', 'Boston Celtics'), 33),
 (('2010', 'Los Angeles Lakers'), 32),
 (('2008', 'Los Angeles Lakers'), 32),
 (('2015', 'Los Angeles Clippers'), 32),
 (('2013', 'San Antonio Spurs'), 31),
 (('2016', 'Cleveland Cavaliers'), 31),
 (('2013', 'Oklahoma City Thunder'), 31),
 (('2014', 'Chicago Bulls'), 31),
 (('2012', 'Oklahoma City Thunder'), 31),
 (('2014', 'San Antonio Spurs'), 31),
 (('2009', 'Boston Celtics'), 31),
 (('2010', 'Boston Celtics'), 30),
 (('2009', 'Orlando Magic'), 30),
 (('2015', 'Cleveland Cavaliers'), 30),
 (('2014', 'Golden State Warriors'), 29),
 (('2012', 'Indiana Pacers'), 29),
 (('2014', 'Memphis Grizzlies'), 29),
 (('2016', 'Oklahoma City Thunder'), 29),
 (('2010', 'Orlando Magic'), 29),
 (('201

Junto los rdd de mejor local y mejor visitante y ordeno en función del número de victorias.

In [41]:
mejor_equipo = mejor_local.join(mejor_visitante).map(lambda x: (x[0], (int(x[1][0]) + int(x[1][1]))))
mejor_equipo.sortBy(lambda x: -x[1]).collect()

[(('2015', 'Golden State Warriors'), 88),
 (('2016', 'Golden State Warriors'), 87),
 (('2013', 'Miami Heat'), 85),
 (('2008', 'Boston Celtics'), 84),
 (('2012', 'San Antonio Spurs'), 82),
 (('2009', 'Los Angeles Lakers'), 81),
 (('2012', 'Miami Heat'), 79),
 (('2012', 'Oklahoma City Thunder'), 79),
 (('2008', 'Los Angeles Lakers'), 77),
 (('2016', 'Cleveland Cavaliers'), 77),
 (('2009', 'Cleveland Cavaliers'), 76),
 (('2013', 'San Antonio Spurs'), 73),
 (('2014', 'San Antonio Spurs'), 73),
 (('2016', 'San Antonio Spurs'), 72),
 (('2010', 'Los Angeles Lakers'), 71),
 (('2009', 'Orlando Magic'), 70),
 (('2015', 'Cleveland Cavaliers'), 70),
 (('2016', 'Toronto Raptors'), 68),
 (('2012', 'Los Angeles Clippers'), 68),
 (('2013', 'Indiana Pacers'), 67),
 (('2010', 'Orlando Magic'), 67),
 (('2010', 'Boston Celtics'), 66),
 (('2015', 'Atlanta Hawks'), 66),
 (('2013', 'Oklahoma City Thunder'), 66),
 (('2015', 'San Antonio Spurs'), 66),
 (('2012', 'Chicago Bulls'), 65),
 (('2014', 'Los Angeles C

Se observa que Golden state Warriors son los mejores en 2015 y 2016, San Antonio Spurs en 2014, Miami Heat en 2013, etc.

##  Décima pregunta: Escribe la expresión regular correcta que sólo macheen los teléfonos y el correo del siguiente texto.

Los teléfonos vienen expresados en la forma XXX XX XX XX. Siendo X un número en todo caso, por ello establezo la siguiente expresión.

In [42]:
expresion_telefonos = re.compile('(\d{3}\s\d{2}\s\d{2}\s\d{2})')

El correo electrónico tiene la forma XXXXX@XXX.com. La cadena de texto anterior al @ está formada por 15 caractéres sin espacios en blanco. La cádena de texto posterior al @ está formada por 9 caractéres, un punto y otros 3 caractéres sin espacios en blanco.

In [43]:
expresion_email = re.compile('(\S{15}\@\S{9}\.\S{3})')

Genero una variable que será el texto.

In [44]:
texto = 'Si eres cliente y necesitas información sobre tus posiciones, productos o realizar operaciones: Desde España. Desde el extranjero. Banca telefónica en castellano. Bandera castellano. 902 13 23 13. Banca telefónica en catalán. Bandera catalana. 902 88 30 08. Banca telefónica en inglés. Bandera inglesa. 902 88 88 35. O por correo electrónico a atencioncliente@bankinter.com'

Con findall() lo que hago es que busque todas las coincidencias dentro del texto.

In [45]:
expresion_telefonos.findall(texto)

['902 13 23 13', '902 88 30 08', '902 88 88 35']

In [46]:
expresion_email.findall(texto)

['atencioncliente@bankinter.com']

# Décimo primera pregunta: Histograma de los puntos de los visitantes

__OBJETIVO:__ Replicar la función histogram de los rdd, esta nos ofrece una lista de tuplas formadas por los cortes de las columnas y las frecuencias de cada columna.

Realizo un mapeo sobre el rdd liga_parsed y me quedo con los puntos de los equipos visitantes.

In [47]:
visitor_points = liga_parsed.map(lambda x: int(x[6]))

Hago un collect y almaceno la lista en una variable.

In [48]:
x = visitor_points.collect()

Ordeno los elementos de la lista.

In [49]:
x.sort()

Calculo los cuaertiles.

In [50]:
# 25%

x[int(len(x)*.25)]

91

In [51]:
# 50%

x[int(len(x)*.50)]

98

In [52]:
# 75%

x[int(len(x)*.75)]

107

Defino una función para aplicar luego mediante un map, me clasifica los puntos en los cuartiles.

In [53]:
def cuartiles(puntos):
    if puntos <= 98:
        if puntos <= 91:
            return (1)
        else:
            return (2)
    else:
        if puntos <= 107:
            return (3)
        else:
            return (4) 

Aplico el map con la función cuartiles, genero también un contador, de esta forma tendremos un rdd con tuplas que tendrán como key o label el cuartil correspondiente y la frecuencia como valor.

In [54]:
visitor_points.map(lambda x: (cuartiles(x),1)).reduceByKey(lambda y,z: y + z).collect()

[(1, 3600), (2, 2877), (3, 3446), (4, 2974)]

In [57]:
sc.stop()